# Exercise 3: Word Classification

In [1]:
import nltk
nltk.download('names', quiet = True)

True

In [2]:
files=nltk.corpus.names.fileids()
# print(files)
female=nltk.corpus.names.words('female.txt')
male=nltk.corpus.names.words('male.txt')

In [3]:
male[:20]
female[:20]

['Abagael',
 'Abagail',
 'Abbe',
 'Abbey',
 'Abbi',
 'Abbie',
 'Abby',
 'Abigael',
 'Abigail',
 'Abigale',
 'Abra',
 'Acacia',
 'Ada',
 'Adah',
 'Adaline',
 'Adara',
 'Addie',
 'Addis',
 'Adel',
 'Adela']

In [4]:
print('male ='+ str(len(male)))
print('female =' + str(len(female)))

male =2943
female =5001


In [5]:
import random
random.seed(0)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
)


### Make sure that the data is balanced. One label should not be x1.5 the other one.

In [7]:
female = list(female)
random.shuffle(female)
female = female[:len(male)]
words = female + list(male)
labels = [0] * len(female) + [1] * len(male)


In [8]:
len(words), len(labels)

(5886, 5886)

### Create features that might be an indicator of a name’s gender

In [9]:
import random
from nltk.corpus import names
import nltk

In [10]:
def gender_features(word):
    return {'last_letter':word[-1]}
  
# preparing a list of examples and corresponding class labels.
labeled_names = ([(name, 'male') for name in names.words('male.txt')]+
             [(name, 'female') for name in names.words('female.txt')])
  
random.shuffle(labeled_names)
  
featuresets = [(gender_features(n), gender) 
               for (n, gender)in labeled_names]
  
train_set, test_set = featuresets[500:], featuresets[:500]
  
classifier = nltk.NaiveBayesClassifier.train(train_set)
  
print(classifier.classify(gender_features('josegerald')))
  
# output should be 'male'
print(nltk.classify.accuracy(classifier, train_set))

classifier.show_most_informative_features(20)


def word_features(word):
    return {
        '3-suffix': word[-3:],
        '3-prefix': word[:3]
}

word_features = [
    word_features(w)
    for w in words
]

word_features[:10]

male
0.7647770016120365
Most Informative Features
             last_letter = 'a'            female : male   =     35.6 : 1.0
             last_letter = 'k'              male : female =     30.4 : 1.0
             last_letter = 'f'              male : female =     16.0 : 1.0
             last_letter = 'v'              male : female =     11.2 : 1.0
             last_letter = 'p'              male : female =     10.6 : 1.0
             last_letter = 'd'              male : female =     10.3 : 1.0
             last_letter = 'o'              male : female =      8.8 : 1.0
             last_letter = 'm'              male : female =      8.6 : 1.0
             last_letter = 'r'              male : female =      7.4 : 1.0
             last_letter = 'g'              male : female =      5.5 : 1.0
             last_letter = 'w'              male : female =      5.1 : 1.0
             last_letter = 't'              male : female =      4.4 : 1.0
             last_letter = 's'              male :

[{'3-suffix': 'rci', '3-prefix': 'Dar'},
 {'3-suffix': 'cka', '3-prefix': 'Eri'},
 {'3-suffix': 'lie', '3-prefix': 'Ell'},
 {'3-suffix': 'ena', '3-prefix': 'Cor'},
 {'3-suffix': 'ine', '3-prefix': 'Fra'},
 {'3-suffix': 'nor', '3-prefix': 'Ele'},
 {'3-suffix': 'yne', '3-prefix': 'Fra'},
 {'3-suffix': 'nta', '3-prefix': 'Ata'},
 {'3-suffix': 'ite', '3-prefix': 'Ben'},
 {'3-suffix': 'lly', '3-prefix': 'Lol'}]

### Split the dataset to 75% 10%, 15% for train, validation, and test respectively

In [11]:
train_features, test_features, train_labels, test_labels = \
    train_test_split(word_features, labels, test_size=0.15, stratify=labels, random_state=0)

train_features, val_features, train_labels, val_labels = \
    train_test_split(train_features, train_labels, test_size=1/8, stratify=train_labels, random_state=0)

len(train_features), len(test_features), len(val_features)

(4377, 883, 626)

### Use a Random Forest to do the classification

In [12]:
dv = DictVectorizer(sparse=False)
train_vectors = dv.fit_transform(train_features)
val_vectors = dv.transform(val_features)
test_vectors = dv.transform(test_features)

dv.feature_names_[:10], dv.feature_names_[-10:]


nb = BernoulliNB()
nb.fit(train_vectors, train_labels)

train_predict = nb.predict(train_vectors)
val_predict = nb.predict(val_vectors)
test_predict =  nb.predict(test_vectors)

### Calculate the ROC AUC, precision, recall, accuracy, and confusion matrix for each dataset split


In [13]:
roc_auc_score(train_labels, train_predict)

print(classification_report(train_labels, train_predict))

confusion_matrix(train_labels, train_predict)


roc_auc_score(val_labels, val_predict)


              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2188
           1       0.92      0.90      0.91      2189

    accuracy                           0.91      4377
   macro avg       0.91      0.91      0.91      4377
weighted avg       0.91      0.91      0.91      4377



0.8099041533546326

### Show the top 10 and the bottom 10 features based on importance


In [14]:
best_neg = nb.feature_log_prob_[0].argsort()
best_pos = nb.feature_log_prob_[1].argsort()

[dv.feature_names_[idx] for idx in best_neg[-10:][::-1]]

[dv.feature_names_[idx] for idx in best_pos[-10:][::-1]]

['3-suffix=ton',
 '3-suffix=ard',
 '3-prefix=Mar',
 '3-prefix=Bar',
 '3-suffix=son',
 '3-suffix=ley',
 '3-suffix=ick',
 '3-suffix=ter',
 '3-prefix=Wil',
 '3-suffix=ert']